In [3]:
import numpy as np
import matplotlib.pyplot as plt
def myfft2(A): 
    return np.fft.fftshift(np.fft.fft2(np.fft.fftshift(A)))
def myifft2(A): 
    return np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(A)))
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
def base_model():
    model = Sequential()
    model.add(Dense(dim**2, input_dim= 2*dim ** 2, activation='linear', use_bias=False))  
    model.compile(loss='MSE', optimizer='adam', metrics=['accuracy'])
    return model

In [2]:
datasets=np.load('curated_data/speckle_datasets.npy') # If generating your own data, change to 'user_created_data/speckle_datasets.npy'

The next cell 

In [ ]:


m1 = 1 ; m2 = 3

xmax = 10
dim = 32
dx = 2*xmax/dim ; x = np.linspace(-xmax,xmax-dx,dim)

X,Y=np.meshgrid(x,x)
R2=(X**2+Y**2)
flambda=80
Lens = np.exp(-1j*R2/(flambda/10))


T = np.arctan2(Y,X)
V1 = np.exp(1j*(m1)*T) ; V2 = np.exp(1j*(m2)*T)

LG1=Lens*V1 ; LG2=Lens*V2
plt.imshow(np.angle(np.hstack((LG1,LG2))),cmap='twilight')
plt.axis('off')
plt.show()


def vort_THIS(A,LGA,LGB):
    length=A.shape[0]
    out=np.zeros((length,A.shape[1],A.shape[2]*2))
    for i in range(length):
        A1=np.abs(myfft2(np.exp(1j*A[i]*np.pi/2)*LGA)) ; A2=np.abs(myfft2(np.exp(1j*A[i]*np.pi/2)*LGB))
        A1=(A1-np.amin(A1))/np.amax(A1-np.amin(A1))*2-1    ; A2=(A2-np.amin(A2))/np.amax(A2-np.amin(A2))*2-1
        #A1=A1/np.sum(A1)
        out[i]=np.hstack((A1,A2))
    return out

In [ ]:
# Create tensorflow models. Store histories

imax=10000 # Number of images 

models=[] # Stores generated models
histories=[]
for i in range(10):
    ds=datasets[i]
    for j in range(imax): # Have to normalize [0,1]
        ds[j]=(ds[j]-np.amin(ds[j]))/np.amax(ds[j]-np.amin(ds[j]))
    